In [ ]:
# ================================================================
# ETAPA 03: PRÉ-PROCESSAMENTO - CONFIGURAÇÃO DO AMBIENTE SPARK
# ================================================================

# Fecha qualquer sessão Spark anterior para evitar conflitos
# Importante fazer isso antes de configurar um novo ambiente
try:
    spark.stop()
except:
    pass

# Instalação e configuração do ambiente Java + PySpark
# Java 17 é necessário para compatibilidade com PySpark 3.5.1
print("Configurando ambiente Java + PySpark...")
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless -qq
!pip -q install -U pyspark==3.5.1

# Configuração das variáveis de ambiente para Java
# JAVA_HOME aponta para a instalação do Java
# PATH é atualizado para incluir os binários do Java
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"]  = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

# Criação da sessão Spark
# SparkSession é o ponto de entrada para funcionalidades do Spark
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("eixo05-preprocess")  # Nome da aplicação Spark
         .getOrCreate())

print("Spark configurado com sucesso!")
print(f"Versão do Spark: {spark.version}")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Spark OK -> 3.5.1


In [ ]:
# ================================================================
# CONFIGURAÇÃO DE CAMINHOS E ACESSO AO GOOGLE DRIVE
# ================================================================

# Monta o Google Drive (força remontagem se necessário)
# O Google Drive serve como storage persistente entre sessões
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# Define caminhos para os dados de entrada e saída
# base_path: diretório raiz onde ficam todos os dados do projeto
# csv_path: arquivo CSV gerado na etapa de coleta de dados
base_path = "/content/drive/MyDrive/Eixo_05/dados/"
csv_path  = base_path + "dataset.csv"

print("Google Drive montado com sucesso!")
print(f"Caminho base: {base_path}")
print(f"Arquivo de entrada: {csv_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Base path: /content/drive/MyDrive/Eixo_05/dados/


In [ ]:
# ================================================================
# IMPORTS E FUNÇÕES AUXILIARES PARA PRÉ-PROCESSAMENTO DE TEXTO
# ================================================================

# Imports das bibliotecas PySpark necessárias
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lower, regexp_replace, concat
from pyspark.ml.feature import (
    StringIndexer, RegexTokenizer, StopWordsRemover,
    HashingTF, IDF, Word2Vec, MinMaxScaler, NGram, CountVectorizer
)

def limpar_texto(df: DataFrame, coluna="review"):
    """
    Função para limpeza de texto usando PySpark
    
    Etapas de limpeza:
    1. Remove tags HTML (ex: <br>, <p>, etc.)
    2. Remove caracteres especiais, mantendo apenas letras e espaços
    3. Colapsa múltiplos espaços em um único espaço
    4. Converte todo o texto para minúsculas
    
    Args:
        df: DataFrame do PySpark
        coluna: nome da coluna de texto a ser limpa
    
    Returns:
        DataFrame com a coluna de texto limpa
    """
    # Remove tags HTML simples (qualquer texto entre < e >)
    df = df.withColumn(coluna, regexp_replace(col(coluna), r"<[^>]+>", ""))
    
    # Remove caracteres especiais, mantendo apenas letras e espaços
    df = df.withColumn(coluna, regexp_replace(col(coluna), r"[^A-Za-z ]+", " "))
    
    # Colapsa múltiplos espaços consecutivos em um único espaço
    df = df.withColumn(coluna, regexp_replace(col(coluna), r"\s+", " "))
    
    # Converte tudo para minúsculas para normalização
    df = df.withColumn(coluna, lower(col(coluna)))
    
    return df

In [ ]:
# ================================================================
# CARREGAMENTO E PRÉ-PROCESSAMENTO INICIAL DOS DADOS
# ================================================================

# 1) CARREGAMENTO DO DATASET
# Lê o CSV gerado na etapa de coleta, selecionando apenas colunas necessárias
# escape="\"" trata aspas duplas corretamente no CSV
print("Carregando dataset CSV...")
reviews = spark.read.csv(csv_path, header=True, escape="\"").select("sentiment", "review")

# 2) LIMPEZA DE DADOS NULOS
# Remove linhas com valores nulos nas colunas essenciais
# Essencial para evitar erros no processamento posterior
print("Removendo valores nulos...")
reviews = reviews.na.drop(subset=["sentiment", "review"])

# 3) INDEXAÇÃO DE LABELS
# Converte categorias de texto ("positive", "negative") em índices numéricos
# handleInvalid="skip" pula valores inválidos sem travar o processamento
print("Convertendo labels categóricos para numéricos...")
indexer = StringIndexer(inputCol="sentiment", outputCol="label", handleInvalid="skip")
df = indexer.fit(reviews).transform(reviews)

# 4) LIMPEZA DE TEXTO
# Aplica a função de limpeza definida anteriormente
print("Limpando texto das avaliações...")
df = limpar_texto(df, coluna="review")

# 5) TOKENIZAÇÃO
# Divide o texto em palavras individuais usando regex
# pattern=r"\W+" divide por qualquer sequência de caracteres não-alfanuméricos
print("Tokenizando texto...")
df = RegexTokenizer(inputCol="review", outputCol="words", pattern=r"\W+").transform(df)

# 6) REMOÇÃO DE STOPWORDS
# Remove palavras comuns sem valor semântico (the, and, is, etc.)
# caseSensitive=False para tratar palavras independente de maiúsculas/minúsculas
print("Removendo stopwords...")
df = StopWordsRemover(inputCol="words", outputCol="filtered", caseSensitive=False).transform(df)

print("Pré-processamento inicial concluído!")
print(f"Total de registros processados: {df.count():,}")

In [ ]:
# ================================================================
# FEATURIZAÇÃO HTF (HASHING TERM FREQUENCY) E TF-IDF
# ================================================================

print("Iniciando processo de featurização...")

# === FEATURIZAÇÃO HTF (HASHING TERM FREQUENCY) ===
# HTF usa função hash para mapear termos em features de tamanho fixo
# Vantagens: rápido, não precisa armazenar vocabulário
# numFeatures: tamanho do vetor de features (2^18 = ~262k features)
print("Aplicando HTF (Hashing Term Frequency)...")
htf = HashingTF(inputCol="filtered", outputCol="rawfeatures", numFeatures=1<<18)
htf_df = htf.transform(df)

# Seleciona e renomeia colunas para padronização
HTFfeaturizedData = (
    htf_df.select("sentiment", "review", "label", "rawfeatures")
          .withColumnRenamed("rawfeatures", "features")
)

print("HTF aplicado com sucesso!")

# === FEATURIZAÇÃO TF-IDF (TERM FREQUENCY - INVERSE DOCUMENT FREQUENCY) ===
# TF-IDF combina frequência de termos com raridade no corpus
# Inclui unigramas (palavras individuais) + bigramas (pares de palavras)

# 1) GERAÇÃO DE BIGRAMAS
# N-gramas capturam relações entre palavras adjacentes
# Ex: ["very", "good"] vira "very good"
print("Gerando bigramas...")
ngram = NGram(n=2, inputCol="filtered", outputCol="bigrams")
df_ng = ngram.transform(df)

# 2) CONCATENAÇÃO DE UNIGRAMAS + BIGRAMAS
# Combina palavras individuais com pares de palavras
# Aumenta a expressividade das features
print("Combinando unigramas e bigramas...")
df_tokens = df_ng.withColumn("tokens_12", concat("filtered", "bigrams"))

# 3) COUNT VECTORIZER
# Constrói vocabulário controlado e conta frequências de termos
# minDF=2: remove termos que aparecem em menos de 2 documentos (reduz ruído)
# vocabSize: limita tamanho do vocabulário para controle de memória
print("Construindo vocabulário e contando termos...")
cv = CountVectorizer(inputCol="tokens_12", outputCol="rawfeatures", minDF=2, vocabSize=1<<18)
cv_model = cv.fit(df_tokens)
cv_df = cv_model.transform(df_tokens)

# 4) APLICAÇÃO DO IDF
# IDF reduz peso de termos muito comuns e aumenta peso de termos raros
# Melhora a capacidade discriminativa das features
print("Aplicando IDF (Inverse Document Frequency)...")
idf = IDF(inputCol="rawfeatures", outputCol="features")
idf_model = idf.fit(cv_df)
TFIDFfeaturizedData = idf_model.transform(cv_df) \
    .select("sentiment", "review", "label", "features")

print("TF-IDF aplicado com sucesso!")
print(f"Tamanho do vocabulário: {len(cv_model.vocabulary):,} termos")

In [ ]:
# ================================================================
# FEATURIZAÇÃO WORD2VEC COM ESCALONAMENTO
# ================================================================

# === FEATURIZAÇÃO WORD2VEC ===
# Word2Vec cria embeddings vetoriais densos que capturam semântica
# Palavras similares ficam próximas no espaço vetorial

print("Aplicando Word2Vec...")

# Parâmetros do Word2Vec:
# vectorSize: dimensão dos vetores de embedding (250 dimensões)
# minCount: ignora palavras que aparecem menos de 5 vezes (reduz ruído)
# seed: garante reprodutibilidade dos resultados
w2v = Word2Vec(
    inputCol="filtered", 
    outputCol="features", 
    vectorSize=250, 
    minCount=5, 
    seed=42
)

# Treina o modelo Word2Vec e aplica transformação
w2v_model = w2v.fit(df)
w2v_df = w2v_model.transform(df)

print("Word2Vec aplicado com sucesso!")

# === ESCALONAMENTO MIN-MAX ===
# Normaliza os vetores Word2Vec para escala [0,1]
# Importante para algoritmos sensíveis à escala (como SVM)

print("Aplicando escalonamento Min-Max...")

scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(w2v_df)
scaled = scaler_model.transform(w2v_df)

# Seleciona colunas finais e renomeia para padronização
W2VfeaturizedData = (
    scaled.select("sentiment", "review", "label", "scaledFeatures")
          .withColumnRenamed("scaledFeatures", "features")
)

print("Escalonamento concluído!")
print(f"Dimensão dos vetores Word2Vec: {w2v.getVectorSize()}")
print(f"Vocabulário Word2Vec: {len(w2v_model.getVectors().collect()):,} palavras")

In [ ]:
# ================================================================
# SALVAMENTO DOS DATASETS FEATURIZADOS
# ================================================================

print("Salvando datasets featurizados em formato Parquet...")

# === SALVAMENTO EM PARQUET ===
# Parquet é eficiente para armazenamento de dados estruturados
# mode("overwrite") substitui arquivos existentes

# Salva dataset com features HTF
HTFfeaturizedData.write.mode("overwrite").parquet(base_path + "HTFfeaturizedData")
print(f"HTF salvo em: {base_path}HTFfeaturizedData")

# Salva dataset com features TF-IDF
# select() garante que apenas colunas necessárias sejam salvas
TFIDFfeaturizedData.select("sentiment","review","label","features") \
    .write.mode("overwrite").parquet(base_path + "TFIDFfeaturizedData")
print(f"TF-IDF salvo em: {base_path}TFIDFfeaturizedData")

# Salva dataset com features Word2Vec
W2VfeaturizedData.write.mode("overwrite").parquet(base_path + "W2VfeaturizedData")
print(f"Word2Vec salvo em: {base_path}W2VfeaturizedData")

# === ATRIBUIÇÃO DE NOMES PARA RASTREAMENTO ===
# Adiciona atributo 'name' aos DataFrames para identificação posterior
# Útil para logging e debugging nas próximas etapas
HTFfeaturizedData.name   = "HTFfeaturizedData"
TFIDFfeaturizedData.name = "TFIDFfeaturizedData"
W2VfeaturizedData.name   = "W2VfeaturizedData"

print("\n" + "="*60)
print("RESUMO DOS DATASETS GERADOS:")
print("="*60)
print("1. HTF (Hashing Term Frequency)")
print("   - Features esparsas baseadas em hashing")
print("   - Rápido processamento, sem vocabulário armazenado")
print()
print("2. TF-IDF (Term Frequency - Inverse Document Frequency)")
print("   - Combina unigramas + bigramas")
print("   - Vocabulário controlado com minDF=2")
print()
print("3. Word2Vec")
print("   - Embeddings semânticos de 250 dimensões")
print("   - Escalados com Min-Max [0,1]")
print("="*60)

Salvo em:
 - /content/drive/MyDrive/Eixo_05/dados/HTFfeaturizedData
 - /content/drive/MyDrive/Eixo_05/dados/TFIDFfeaturizedData
 - /content/drive/MyDrive/Eixo_05/dados/W2VfeaturizedData


In [ ]:
# ================================================================
# VERIFICAÇÃO FINAL DOS DATASETS
# ================================================================

# Sanity check: verifica se todos os datasets têm o mesmo número de registros
# Importante para garantir consistência antes de prosseguir para o ML

print("Realizando verificação final dos datasets...")
print()

# Conta registros em cada dataset
htf_count = HTFfeaturizedData.count()
tfidf_count = TFIDFfeaturizedData.count()
w2v_count = W2VfeaturizedData.count()

print("CONTAGEM DE REGISTROS:")
print(f"HTF:     {htf_count:,} registros")
print(f"TF-IDF:  {tfidf_count:,} registros") 
print(f"Word2Vec: {w2v_count:,} registros")

# Verifica se todas as contagens são iguais
if htf_count == tfidf_count == w2v_count:
    print(f"\nSUCESSO: Todos os datasets têm {htf_count:,} registros!")
    print("Os dados estão prontos para a etapa de Machine Learning.")
else:
    print(f"\nATENÇÃO: Inconsistência no número de registros!")
    print("Revisar o processamento antes de prosseguir.")

print(f"\nPróxima etapa: Executar 'aprendizado_maquina.ipynb'")

Contagens: 50000 50000 50000
